In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [30]:
# Define data transformations with augmentation
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.Resize((224, 224)),  # Resize to match ResNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test = datasets.ImageFolder(root='4B-3D-Uni/test', transform=transform)
# Create data loaders testing
test_loader = DataLoader(test, batch_size=16, shuffle=False)

print(f"Testing set size: {len(test)} images")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

Testing set size: 400 images


In [32]:
def evaluate_model(model, data_loader):
    """Evaluate model on the provided data loader"""
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    avg_loss = running_loss / len(data_loader)
    return accuracy, avg_loss

In [49]:
# Final evaluation on test set
from torchvision.models import resnet18, ResNet18_Weights
# Initialize the ResNet18 model (without pre-trained weights)
model = resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

# Load fine-tuned weights
model.load_state_dict(torch.load('saved_models/4B-3D-Non_best_model.pth', weights_only=True))

# Move the model to GPU
model = model.to(device)

test_accuracy, test_loss = evaluate_model(model, test_loader)
print(f'\nFinal Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Final Test Loss: 0.6473, Test Accuracy: 78.25%
